In [1]:
import mitiq

## Challenge 1: define a benchmarking circuit

Create a Clifford+T circuit


In [2]:
ct = mitiq.benchmarks.generate_random_clifford_t_circuit(4, 8, 5, 3)
print(ct)

                  ┌──┐
0: ───@───S───S────H─────T───S───@───S───@───
      │                          │       │
1: ───┼───────X────@─────────────@───────┼───
      │       │    │                     │
2: ───@───S───@────┼H────────────────────@───
                   │
3: ───H───T────────X─────T───────────────────
                  └──┘


## Challenge 2: convert the ciruit to qiskit

By default, benchmarking circuits are instantiated as `cirq.Circuit` objects. Convert the circuit to a `qiskit.QuantumCircuit` object.


In [5]:
qiskit_ct = mitiq.interface.convert_from_mitiq(ct, "qiskit")
print(qiskit_ct)

          ┌───┐┌───┐┌───┐┌───┐┌───┐   ┌───┐   
q_0: ──■──┤ S ├┤ S ├┤ H ├┤ T ├┤ S ├─■─┤ S ├─■─
       │  └───┘├───┤└───┘└───┘└───┘ │ └───┘ │ 
q_1: ──┼───────┤ X ├──■─────────────■───────┼─
       │  ┌───┐└─┬─┘  │  ┌───┐              │ 
q_2: ──■──┤ S ├──■────┼──┤ H ├──────────────■─
     ┌───┐├───┤     ┌─┴─┐├───┤                
q_3: ┤ H ├┤ T ├─────┤ X ├┤ T ├────────────────
     └───┘└───┘     └───┘└───┘                


## Challenge 3: define a pauli string


In [9]:
ps = mitiq.PauliString("IY")
print(ps)

Y(q(1))


## Challenge 4: define simultaneously measurable pauli strings

Define a pauli string that can be measured simultaneously with the above `PauliString` object.


In [11]:
ps2 = mitiq.PauliString("XII")

ps.can_be_measured_with(ps2)

True

## Challenge 5: find the pauli string with a specific matrix

What pauli string defines the following matrix

$$
\begin{bmatrix}
    0 & 0  & 0 & -i \\
    0 & 0  & i & 0  \\
    0 & -i & 0 & 0  \\
    i & 0  & 0 & 0  \\
\end{bmatrix}
$$


In [17]:
ps = mitiq.PauliString("XY")
ps.matrix()

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
       [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j]])

## Challenge 6: measure pauli strings in a circuit

using the clifford+T circuit defined in challenge 1, measure two pauli strings in it.


In [20]:
paulis = [mitiq.PauliString("XIZI"), mitiq.PauliString("YYXI")]

obs = mitiq.Observable(*paulis)

obs.measure_in(ct)

[                  ┌──┐
0: ───@───S───S────H─────T───S───@───S───────@───X^0.5────M───
      │                          │           │            │
1: ───┼───────X────@─────────────@───X^0.5───┼────────────M───
      │       │    │                         │            │
2: ───@───S───@────┼H────────────────────────@───Y^-0.5───M───
                   │
3: ───H───T────────X─────T────────────────────────────────────
                  └──┘,
                   ┌──┐
0: ───@───S───S────H─────T───S───@───S───@───Y^-0.5───M───
      │                          │       │            │
1: ───┼───────X────@─────────────@───────┼────────────┼───
      │       │    │                     │            │
2: ───@───S───@────┼H────────────────────@────────────M───
                   │
3: ───H───T────────X─────T────────────────────────────────
                  └──┘]

## Challenge 7: measure pauli strings in a circuit simultaneously

Define two pauli strings that can be simultaneously measured.


In [21]:
paulis = [mitiq.PauliString("XIZI"), mitiq.PauliString("IXIY")]

obs = mitiq.Observable(*paulis)

obs.measure_in(ct)

[                  ┌──┐
0: ───@───S───S────H─────T───S───────@───S────────@───Y^-0.5───M───
      │                              │            │            │
1: ───┼───────X────@─────────────────@───Y^-0.5───┼────────────M───
      │       │    │                              │            │
2: ───@───S───@────┼H─────────────────────────────@────────────M───
                   │                                           │
3: ───H───T────────X─────T───X^0.5─────────────────────────────M───
                  └──┘]

## Challenge 8: create a `MeasurementResult` object

You've run an experiment, and received the following dictionary back with the counts. Convert it to a `mitiq.MeasurementResult` object.

```py
{
    "0001": 435,
    "0010": 324,
    "0100": 120,
    "1001": 70,
    "0110": 51
}
```


In [28]:
bitstring_counts = {
    "0001": 70,
    "0010": 51,
    "0100": 68,
    "1001": 437,
    "0110": 374
}
mr = mitiq.MeasurementResult.from_counts(bitstring_counts)

## Challenge 9: convert counts to a probability distribution

instead of a dictionary mapping bitstrings to counts, create a dictionary mapping bitstrings to probabilities of measuring said bitstring. Test your answer by comparing to the `prob_distribution` method of your `MeasurementResult` object.


In [29]:
total_count = sum(bitstring_counts.values())
prob = {bits: count / total_count for bits, count in bitstring_counts.items()}

prob == mr.prob_distribution()

True

## Challenge 10: Post selecting with a symmetry condition

Select (programatically) bitstrings which are palindromes.


In [35]:
from mitiq.rem import post_select # can remove

nmr = post_select(mr, lambda bitstring: bitstring == bitstring[::-1])

## Challenge 11: total variation distance

Compute the [total variation distance](https://en.wikipedia.org/wiki/Total_variation_distance_of_probability_measures) between the pre and post-selected measurement results.


In [42]:
import itertools

def generate_bitstrings(length):
    return [''.join(bits) for bits in itertools.product('01', repeat=length)]

pre_dist = mr.prob_distribution()
post_dist = nmr.prob_distribution()

tvd = 0
for bitstring in generate_bitstrings(4):
    tvd += abs(pre_dist.get(bitstring, 0) - post_dist.get(bitstring, 0))

tvd /= 2

print(tvd)

0.18900000000000003


## Challenge 12: global folding

Fold the following circuit globally. That is, construct a new circuit whose operation is $UU^\dag U$ where $U$ is the operation of the entire circuit.


In [54]:
circuit = mitiq.benchmarks.generate_rb_circuits(2, 1)[0]
print(circuit)

0: ───Y─────X───@───Y^0.5───X^0.5───X^0.5────Y^-0.5───@───X^0───────────────────────
                │                                     │
1: ───X^0───────@───Y───────X^0.5───Y^-0.5───X^0.5────@───X^-0.5───Y^-0.5───X^0.5───


In [55]:
mitiq.zne.scaling.fold_global(circuit, scale_factor=3)

0: ───Y─────X───@───Y^0.5───X^0.5───X^0.5────Y^-0.5───@───X^0────────────────────────────────────────X^0─────@───Y^0.5────X^-0.5───X^-0.5───Y^-0.5───@───X───Y─────Y─────X───@───Y^0.5───X^0.5───X^0.5────Y^-0.5───@───X^0───────────────────────
                │                                     │                                                      │                                       │                       │                                     │
1: ───X^0───────@───Y───────X^0.5───Y^-0.5───X^0.5────@───X^-0.5───Y^-0.5───X^0.5───X^-0.5───Y^0.5───X^0.5───@───X^-0.5───Y^0.5────X^-0.5───Y────────@───────X^0───X^0───────@───Y───────X^0.5───Y^-0.5───X^0.5────@───X^-0.5───Y^-0.5───X^0.5───

## Challenge 13: non-integer folding

Using a folding technique to increase the circuits depth by 2.


In [56]:
mitiq.zne.scaling.fold_gates_at_random(circuit, scale_factor=2)

0: ───Y─────X─────X─────X───@───@───@───Y^0.5───Y^-0.5───Y^0.5────X^0.5───X^0.5────X^-0.5───X^0.5────Y^-0.5───Y^0.5───Y^-0.5───@───X^0────────────────────────────────────────
                            │   │   │                                                                                          │
1: ───X^0───X^0───X^0───────@───@───@───Y───────X^0.5────Y^-0.5───Y^0.5───Y^-0.5───X^0.5────X^-0.5───X^0.5─────────────────────@───X^-0.5───X^0.5───X^-0.5───Y^-0.5───X^0.5───

## Challenge 14: gate folding

Suppose all your noise comes from two-qubit gates and you'd like to target them in the folding. Construct a folded circuit where you exclude single qubit gates from being folded.


In [59]:
mitiq.zne.scaling.fold_all(circuit, scale_factor=3, exclude={"single"})

0: ───Y─────X───@───@───@───Y^0.5───X^0.5───X^0.5────Y^-0.5───@───@───@───X^0───────────────────────
                │   │   │                                     │   │   │
1: ───X^0───────@───@───@───Y───────X^0.5───Y^-0.5───X^0.5────@───@───@───X^-0.5───Y^-0.5───X^0.5───

## Challenge 15: Pauli twirling

Generate 100 Pauli twirled variants of a GHZ circuit on 3 qubits. How many of them are unique?


In [127]:
circuit = mitiq.benchmarks.generate_ghz_circuit(3)

from mitiq.pt import generate_pauli_twirl_variants
pts = generate_pauli_twirl_variants(circuit, 100)

len(set(map(str, pts)))

84

## Challenge 16: Pauli twirling gates addition

How many more gates do the variants contain than the original circuit?


In [133]:
num_gates = [len(list(c.all_operations())) for c in pts]

all(g == 11 for g in num_gates)

True

## Challenge 17: Digital dynamical decoupling: XX

Insert an $XX$ sequence in a circuit (you define this).
